In [1]:
import sys
sys.path.append('..')
import atclient
import pandas as pd

## Use the API

### Connect

Create a config.yaml with 3 fields : url, username and password

In [2]:
api = atclient.AtApi(config='../config.yaml')

Token received


### Monitoring / availability check

In [4]:
api.ping()

{'available': True,
 'response_time_ms': 102.65,
 'status_code': 200,
 'timestamp': '2026-02-25T21:28:15.557295'}

### Projects management

Load data to test

In [8]:
df = pd.read_parquet("../data/dataset.parquet")
df.head()

,Unnamed: 0,id,text,label,prob,proba
0,0,1569866966566932480,"Les ENFANTS du Québec, \nLes AÎNÉS du Québec,\...",Autre,0.823412,NaN
1,1,1569919081351942144,Après avoir imposé une politique dystopique zé...,Autre,0.599684,NaN
2,2,1569930968026480640,"@Nyonensis @DIVIZIO1 Pour le coup, si ce qu'il...",Autre,0.941126,NaN
3,3,1569947515465728000,@nadinecerdan @SaiyanBio @nawah_s C'est sous e...,Autre,0.625537,NaN
4,4,1569961852972732416,@L_ThinkTank L'achat de vaccins et d'armes ne ...,Autre,0.103508,NaN


Get available projects

In [7]:
r = api.get_projects()
len(r)

78

Get project slugs

In [13]:
r = api.get_projects_slugs()

(If you want to sluggify a project name, you can use the following function)

In [14]:
from slugify import slugify
slugify("My project name")

'my-project-name'

In [6]:
# Minimal example
# api.add_project("del2", df, "id", ["text"], cols_label=["label"])

# With train/test/valid splits and stratification
r = api.add_project(
    "del3",
    df,
    col_id="id",
    cols_text=["text"],
    cols_label=["label"],
    cols_context=[],
    n_train=500,
    n_test=100,
    n_valid=50,
    language="fr",
    random_selection=True,
)
r

R None


'del3'

Delete a project

In [8]:
api.delete_project("del3")

Project deleted


Export a project (without the models)

In [11]:
api.export_project("v9-articles-echantillonnes-nv-codage", path="./exports", raw_datasets=True)

No test found
Project v9-articles-echantillonnes-nv-codage saved with 1 schemes
HTTPSConnectionPool(host='projet-datalab-gpu-40gb-609227-user.lab.groupe-genes.fr', port=443): Max retries exceeded with url: /static/v9-articles-echantillonnes-nv-codage/v9-articles-echantillonnes-nv-codage_data_all.parquet (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))


In [ ]:
api.export_all(path="../../exports", 
               raw_datasets=False, 
               since=pd.to_datetime("2026-02-01"))

Export all the projects available for a user (for a backup for example)

### Users management

Get all users

In [4]:
len(api.get_users())

113

Add

In [5]:
api.add_user("test3", "test3", "test@test.fr")

User created


Delete

In [6]:
api.delete_user("test3")

User deleted


Add Auth for a project

In [15]:
api.add_auth_user_project("test3", "nouveau-projet-3", "manager")

Auth added to user


Delete Auth for a project

In [16]:
api.delete_auth_user_project("test3", "nouveau-projet-3")

Auth deleted for user


### Get project state

In [7]:
state = api.get_project_state("nouveau-projet-3")

Exception: Error getting project state: Forbidden: Invalid rights

In [ ]:
t = api.get_annotations_data("nouveau-projet-3", "default")
t.head()

In [ ]:
# Get annotations for each split (train / test / valid)
for dataset in ["train", "test", "valid"]:
    t = api.get_annotations_data("nouveau-projet-3", "default", dataset=dataset, verbose=True)
    if t is not None:
        print(f"{dataset}: {len(t)} rows")

In [22]:
t = api.get_annotations_data("nouveau-projet-3", "default")
t.head()

,Unnamed: 0,id,dataset_id,text,limit,labels,user,timestamp,comment
0,0,1569866966566932480,1569866966566932480,"Les ENFANTS du Québec, \nLes AÎNÉS du Québec,\...",1200,Autre,root,2025-02-28 13:39:36.038096,NaN
1,1,1569919081351942144,1569919081351942144,Après avoir imposé une politique dystopique zé...,1200,Autre,root,2025-02-28 13:39:36.038150,NaN
2,2,1569930968026480640,1569930968026480640,"@Nyonensis @DIVIZIO1 Pour le coup, si ce qu'il...",1200,Autre,root,2025-02-28 13:39:36.038174,NaN
3,3,1569947515465728000,1569947515465728000,@nadinecerdan @SaiyanBio @nawah_s C'est sous e...,1200,Autre,root,2025-02-28 13:39:36.038194,NaN
4,4,1569961852972732416,1569961852972732416,@L_ThinkTank L'achat de vaccins et d'armes ne ...,1200,Autre,root,2025-02-28 13:39:36.038212,NaN


### Manage features

Get available feature

In [23]:
r = api.get_features("nouveau-projet-3")
r.keys()

dict_keys([])

Add a SBERT feature on the train dataset

In [24]:
api.add_feature("nouveau-projet-3", "sbert", "sbert")

b'{"detail":"computing sbert, it could take a few minutes","status":"waiting"}'


Get features

In [3]:
df = api.get_features_data("nouveau-projet-3", ["sbert"])
df.head()

,id,sbert__sb001,sbert__sb002,sbert__sb003,sbert__sb004,sbert__sb005,sbert__sb006,sbert__sb007,sbert__sb008,sbert__sb009,...,sbert__sb759,sbert__sb760,sbert__sb761,sbert__sb762,sbert__sb763,sbert__sb764,sbert__sb765,sbert__sb766,sbert__sb767,sbert__sb768
0,1569866966566932480,-0.012411,0.069819,0.017629,0.016337,-0.005500,-0.026143,-0.026683,0.021376,0.018183,...,-0.019169,-0.028596,-0.001524,0.014563,-0.017164,-0.006479,0.050561,0.067910,0.015443,0.025394
1,1569919081351942144,-0.022546,0.042782,0.017333,0.020885,0.006150,-0.027854,0.007987,0.004077,0.012157,...,-0.047567,-0.020814,-0.075401,-0.075062,-0.014331,0.026679,0.014444,0.033841,-0.022563,0.030447
2,1569930968026480640,-0.004847,0.002958,0.048592,0.009563,0.007411,-0.012930,-0.009337,0.007568,0.024637,...,-0.043440,0.036784,-0.016663,-0.030763,-0.006885,-0.014110,0.004118,0.057386,0.037831,-0.009964
3,1569947515465728000,0.009454,0.005232,0.035062,0.005583,0.005241,-0.058031,0.017341,0.031192,0.062633,...,-0.027558,-0.066112,-0.060420,-0.049953,-0.013541,-0.047349,0.004409,0.056827,0.026242,-0.033341
4,1569961852972732416,-0.027911,0.003018,0.042799,-0.023156,-0.013802,-0.019713,0.059756,-0.028939,0.039197,...,-0.026509,-0.009135,-0.007320,-0.028597,-0.009583,0.014385,0.002206,0.035609,0.062151,-0.033790


## Manage scheme

Get available scheme

In [30]:
api.get_schemes("nouveau-projet-3")

{'default': {'labels': ['Autre', 'Règles'], 'kind': 'multiclass'},
 'test': {'labels': ['coucou', 'essai'], 'kind': 'multiclass'}}

Add scheme

In [4]:
api.add_scheme_to_project("nouveau-projet-3","test",["coucou","essai"])

Scheme added to project


Delete scheme

In [3]:
api.delete_scheme_from_project("nouveau-projet-3","test")

Scheme deleted from project


Add a label to a scheme

In [6]:
api.add_label_to_scheme("nouveau-projet-3", "default","test")

Label added to scheme


Remove label from a scheme

In [7]:
api.delete_label_from_scheme("nouveau-projet-3", "default", "test")

Label deleted from scheme


## Manage models

Start fine-tuning a model

In [ ]:
api.get_models("stress")


Stop fine-tuning a model

In [ ]:
api.start_finetune_model("stress","default", "stress", "flaubert/flaubert_base_cased")

Stop a project

In [ ]:
api.stop_finetune_model("stress")